In [1]:
import numpy as np
import nashpy as nash
import axelrod as axl
import random
import warnings
import sqlalchemy as sa
import os
from path import *
import matplotlib.pyplot as plt

In [2]:
noise_probabilities = np.linspace(0, 1, 11)
game_ending_probabilities = np.linspace(0.001, 1 - 0.001, 100)

In [3]:
def get_game(tournament_rep, player_list, prob_of_game_ending, noise):

    """
    A function which runs a tournament of A Prisoner's Dilemma and returns a dictionary containing the following: 'payoff matrix obtained' (an output from the Axelrod tournament function), 'number of tournament repeats' (from 'tournament_rep'), 'probability of game ending' (from 'prob_of_game_ending) and 'noise' (from 'noise'), where:

    'tournament_rep' is a numeric variable stating how many times the tournament should be executed;

    'player_list' is a list containing instances of specific strategy classes obtained from the Axelrod library;

    'prob_of_game_ending' is a numeric variable between 0 and 1 which states the probability of the tournament ending on any specific turn; and

    'noise' is a numeric variable between 0 and 1 as described on https://axelrod.readthedocs.io/en/stable/tutorials/further_topics/noisy_tournaments.html and https://axelrod.readthedocs.io/en/stable/reference/glossary.html#noise.
    """

    tournament = axl.Tournament(
        player_list,
        prob_end=prob_of_game_ending,
        repetitions=tournament_rep,
        noise=noise,
    )
    tournament_results = tournament.play(progress_bar=False)

    mean_payoff_matrix = np.array(tournament_results.payoff_matrix)

    get_game_output_dict = {
        "payoff matrix obtained": mean_payoff_matrix,
        "number of tournament repeats": tournament_rep,
        "probability of game ending": prob_of_game_ending,
        "noise": noise,
    }
    return get_game_output_dict

In [4]:
def get_prob_of_defection(payoff_matrix, support_enumeration=True):

    """
    A function which computes the Nash Equilibria of the game using one of three algorithms and returns a dictionary of the Nash Equilibria, the maximum and minimum probabilities of defection within the equilibria and whether the game could be degenerate. The input variables are:

    'payoff_matrix', a numpy array containing the payoffs obtained for each action of the game; and

    'support_enumeration', a boolean variable stating whether the support enumeration algorithm (if evaluated true) or the vertex enumeration algorithm (if evaluated False) is used to calculate the Nash equilibria.
    """

    game = nash.Game(payoff_matrix, payoff_matrix.transpose())

    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")

        if support_enumeration == True:
            nash_equilibria = list(game.support_enumeration())

        else:
            highlight_numpy_warning = np.seterr(all="warn")
            nash_equilibria = list(game.vertex_enumeration())

    if len(w) == 0:
        warning_message = None

    else:
        warning_message = str([w[i].message for i in range(len(w))])

    if (len(nash_equilibria) == 0) or ("-Inf" in nash_equilibria):
        nash_equilibria = None
        prob_of_defection_in_equilibria = None
        least_prob_of_defection_in_equilibria = None
        greatest_prob_of_defection_in_equilibria = None

    else:
        prob_of_defection_in_equilibria = [
            sigma_1[-1] for sigma_1, _ in nash_equilibria
        ]

        least_prob_of_defection_in_equilibria = min(prob_of_defection_in_equilibria)
        greatest_prob_of_defection_in_equilibria = max(prob_of_defection_in_equilibria)

    get_prob_of_defect_output_dict = {
        "nash equilibria": np.array(nash_equilibria),
        "least prob of defect": least_prob_of_defection_in_equilibria,
        "greatest prob of defect": greatest_prob_of_defection_in_equilibria,
        "warning message": str(warning_message),
    }

    return get_prob_of_defect_output_dict

## 2 player, set 7, noise 0

In [5]:
def rerun(unique_tournament_identifier_start, player_list, game_ending_probs, noise, tournament_rep, player_tournament_set, title_of_plot_file, support_enumeration):
    """
    A function used to rerun a particular player_tournament_set changing one variable (for example, removing the stochastic player, repeating the tournament more times etc.) where:

    'unique_tournament_identifier_start' is the experiment_number from the database;
    
    'player_list' is a list of the specific strategies (including axl.Defector()) who will compete in the tournaments;
    
    'game_ending_probs' is a list of the probabilities of the game finishing after any turn used in the original experiment;

    'noise' is a numeric variable between 0 and 1 stating how much noise should be included (given to 1dp as in the original experiment); 
    'tournament_rep' is a numeric variable stating how many times each tournament should be repeated;

    'player_tournament_set' is the numeric variable which represented this particular group of players in the original experiment;

    'title_of_plot_file.extension' is a string containing the file name which the plot should be saved under; and

    'support_enumeration' is a boolean variable stating whehter the support enumeration algorithm should be used or not

    A plot of the least & greatest probabilities is saved in the same folder as the equivalent plot.
    """
    axl.seed(unique_tournament_identifier_start)
    least_prob_of_defection = []
    greatest_prob_of_defection = []
    num_of_players = len(player_list)
    for probability in game_ending_probs:
        tournament_run = get_game(
            tournament_rep=tournament_rep,
            player_list=player_list,
            prob_of_game_ending=probability,
            noise=noise,
            )
        defection_probs = get_prob_of_defection(
            payoff_matrix=tournament_run["payoff matrix obtained"],
            support_enumeration=support_enumeration,
            )

        least_prob_of_defection.append(defection_probs["least prob of defect"])
        greatest_prob_of_defection.append(defection_probs["greatest prob of defect"])

        unique_tournament_identifier_start += 1
        axl.seed(unique_tournament_identifier_start)

    p = Path(
            "../../images/folk_thm/single_game/"
            + str(num_of_players)
            + "/"
            + str(player_tournament_set)
            + "/"
            + str(noise)
            + "/"
        )
    
    #plot_path = p / title_of_plot_file
    plot_path = Path("../../../") / title_of_plot_file
    graph = plt.figure()
    axes = graph.add_subplot(1, 1, 1)
    axes.set_xlabel("$p =$ the probability of the game ending")
    axes.set_ylabel("probability of defection in equilibria")

    x_values = [
        game_ending_probs,
        game_ending_probs,
        ]
    y_data = [
        least_prob_of_defection,
        greatest_prob_of_defection,
        ]
    colours = ["r", "y"]
    linestyles = ["-", "--"]
    label_list = ["least prob of defection", "greatest prob of defection"]
    for xvalue, data, colour, linestyle, label in zip(
            x_values, y_data, colours, linestyles, label_list
        ):
            axes.plot(xvalue, data, color=colour, linestyle=linestyle, label=label)
    axes.legend()
    graph.savefig(str(plot_path))
    plt.close()

In [6]:
player_list = [axl.MEM2(), axl.Defector()]

In [28]:
rerun_2_7_0 = rerun(unique_tournament_identifier_start=7700, player_list=player_list, game_ending_probs=game_ending_probabilities, noise=0.0, tournament_rep=550, player_tournament_set=7, title_of_plot_file="main_rerun_tourn_rep_550.pdf", support_enumeration=True)

In [33]:
player_list2 = [axl.Ripoff(), axl.Defector()]
rerun_2_14_0 = rerun(unique_tournament_identifier_start=15400, player_list=player_list2, game_ending_probs=game_ending_probabilities, noise=0.0, tournament_rep=1000, player_tournament_set=14, title_of_plot_file="main_rerun_tourn_rep_1000.pdf", support_enumeration=True)

In [34]:
player_list3 = [axl.CollectiveStrategy(), axl.Defector()]
rerun_2_15_0 = rerun(unique_tournament_identifier_start=16500, player_list=player_list3, game_ending_probs=game_ending_probabilities, noise=0.0, tournament_rep=550, player_tournament_set=15, title_of_plot_file="main_rerun_tourn_rep_550.pdf", support_enumeration=True)

In [7]:
player_list4 = [axl.Handshake(), axl.Defector()]
rerun_2_22_0 = rerun(unique_tournament_identifier_start=24200, player_list=player_list4, game_ending_probs=game_ending_probabilities, noise=0.0, tournament_rep=700, player_tournament_set=22, title_of_plot_file="main_rerun_tourn_rep_700.pdf", support_enumeration=True)

In [11]:
player_list5 = [axl.Fortress3(), axl.Defector()]
rerun_2_24_0 = rerun(unique_tournament_identifier_start=26400, player_list=player_list5, game_ending_probs=game_ending_probabilities, noise=0.0, tournament_rep=1500, player_tournament_set=24, title_of_plot_file="main_rerun_tourn_rep_1500.pdf", support_enumeration=True)